# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>
---

**Lab 07**: Structured Streaming with Files 

**Date**: October 10th 2025

**Student Name**: 

**Professor**: Pablo Camarillo Ramirez

In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Examples on Structured Streaming (files)") \
    .master("spark://5cdecfe4c6d6:7077") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("ERROR")
# Optimization (reduce the number of shuffle partitions)
spark.conf.set("spark.sql.shuffle.partitions", "5")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 01:31:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [21]:
from pyspark.sql.functions import count
from pyspark.sql.functions import lit, regexp_extract, col, window, when


#i read from here
df = spark.readStream.format("text").load("/opt/spark/work-dir/data/server_logs")

#i create a new coulmn with just taking wahts in between the [] and get time stamp info
#.strftime("%Y-%m-%d %H:%M:%S")
df_parsed = df.withColumn("level", regexp_extract(col("value"), r"\[(INFO|ERROR|DEBUG|WARNING)\]", 1)) \
    .withColumn("timestamp_str", regexp_extract(col("value"), r"\[(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})\]", 1)) \
    .withColumn("timestamp", col("timestamp_str").cast("timestamp"))


#i wnat to show delta difference between each time an error happens, and if they are happening really often i should give an alert of some kind
errors = df_parsed.filter(col("level") == "ERROR")

windowed_counts = errors.groupBy(
    window(col("timestamp"), "10 seconds", "5 seconds")
).agg(count("*").alias("error_count"))


too_many_errors = 3
alerts = windowed_counts.withColumn(
    "ALERT",
    when(col("error_count") > too_many_errors, lit("Something is probably wrong bro")).otherwise(lit(""))
)

#here i print what i wanna show
query = alerts.writeStream \
    .outputMode("update") \
    .format("console") \
    .start().awaitTermination(300)


